<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

-   Understand the dataset for Chicago Public School level performance
-   Store the dataset in an Db2 database on IBM Cloud instance
-   Retrieve metadata about tables and columns and query data from mixed case columns
-   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**: 

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv">link</a>.

**NOTE**:

For the learners who are encountering issues with loading from .csv in DB2 on Firefox, you can download the .txt files and load the data with those: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.txt">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01"><img src="https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database

The following modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [ ]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [1]:
%load_ext sql

In [66]:
import json

# Specify the file path
file_path = "credentials.txt"

# Read the content of the file
with open(file_path, 'r') as file:
    data_as_str = file.read()

# Parse the content as JSON
try:
    dict_data = json.loads(data_as_str)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

# Access specific values
a=dict_data["connection"]["db2"]["authentication"]

#Replace the placeholder values with your actual Db2 hostname, username, and password:
hostname = dict_data["connection"]["db2"]["hosts"][0]["hostname"]
username = dict_data["connection"]["db2"]["authentication"]["username"].upper()
password = dict_data["connection"]["db2"]["authentication"]["password"]

database_name = dict_data["connection"]["db2"]["database"]
port = dict_data["connection"]["db2"]["hosts"][0]["port"]
security = dict_data["connection"]["cli"]["arguments"][0][4][2:]
cred = f"{username}:{password}@{hostname}:{port}/{database_name}?security={security}"


In [67]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://$cred

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [4]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql SELECT TABSCHEMA,TABNAME, CREATE_TIME FROM SYSCAT.TABLES WHERE TABSCHEMA= :username

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


tabschema,tabname,create_time
DLV46630,EMPLOYEES,2023-12-23 18:03:11.937946
DLV46630,JOB_HISTORY,2023-12-23 18:03:12.049988
DLV46630,JOBS,2023-12-23 18:03:12.210976
DLV46630,DEPARTMENTS,2023-12-23 18:03:12.385413
DLV46630,LOCATIONS,2023-12-23 18:03:12.569588
DLV46630,PETSALE,2023-12-23 18:26:08.491470
DLV46630,PET,2023-12-23 18:26:08.577591
DLV46630,PETRESCUE,2023-12-24 12:51:48.654272
DLV46630,INTERNATIONAL_STUDENT_TEST_SCORES,2023-12-27 17:24:49.406460
DLV46630,INSTRUCTOR,2023-12-28 06:15:20.168858


In [5]:
%sql select * from SYSCAT.TABLES where TABNAME = 'CHICAGO_PUBLIC_SCHOOLS'

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
DLV46630,CHICAGO_PUBLIC_SCHOOLS,DLV46630,U,T,N,None,None,None,None,2023-12-29 08:24:18.864877,2023-12-29 08:24:18.864877,2023-12-29 08:24:18.864877,None,78,15,69,-1,-1,-1,-1,-1,-1,-1,-1,DLV46630SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2023-12-29 08:24:18.864877,0,,None,None,N,DLV46630,,,,2023-12-29,,N,R,N,-1.0,None


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [6]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from syscat.columns where tabname= 'CHICAGO_PUBLIC_SCHOOLS'

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


1
78


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [8]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select colname,typename,length from syscat.columns where tabname= 'CHICAGO_PUBLIC_SCHOOLS' limit 10;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


colname,typename,length
SCHOOL_ID,INTEGER,4
NAME_OF_SCHOOL,VARCHAR,65
ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL,VARCHAR,2
STREET_ADDRESS,VARCHAR,29
CITY,VARCHAR,7
STATE,VARCHAR,2
ZIP_CODE,INTEGER,4
PHONE_NUMBER,VARCHAR,14
LINK,VARCHAR,78
NETWORK_MANAGER,VARCHAR,40


Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "_"?


In [14]:
%sql select * from CHICAGO_PUBLIC_SCHOOLS limit 2;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,elementary__middle__or_high_school,street_address,city,state,zip_code,phone_number,link,network_manager,collaborative_name,adequate_yearly_progress_made_,track_schedule,cps_performance_policy_status,cps_performance_policy_level,healthy_school_certified,safety_icon,safety_score,family_involvement_icon,family_involvement_score,environment_icon,environment_score,instruction_icon,instruction_score,leaders_icon,leaders_score,teachers_icon,teachers_score,parent_engagement_icon,parent_engagement_score,parent_environment_icon,parent_environment_score,average_student_attendance,rate_of_misconducts__per_100_students_,average_teacher_attendance,individualized_education_program_compliance_rate,pk_2_literacy__,pk_2_math__,gr3_5_grade_level_math__,gr3_5_grade_level_read__,gr3_5_keep_pace_read__,gr3_5_keep_pace_math__,gr6_8_grade_level_math__,gr6_8_grade_level_read__,gr6_8_keep_pace_math_,gr6_8_keep_pace_read__,gr_8_explore_math__,gr_8_explore_read__,isat_exceeding_math__,isat_exceeding_reading__,isat_value_add_math,isat_value_add_read,isat_value_add_color_math,isat_value_add_color_read,students_taking__algebra__,students_passing__algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),net_change_explore_and_plan,11th Grade Average ACT (2011),net_change_plan_and_act,college_eligibility__,graduation_rate__,college_enrollment_rate__,college_enrollment,general_services_route,freshman_on_track_rate__,x_coordinate,y_coordinate,latitude,longitude,community_area_number,community_area_name,ward,police_district,location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99,Very Strong,99,Strong,74,Strong,66,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54,Strong,66,Strong,74,Very Strong,84,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [25]:
%sql select count(*) as number\
from CHICAGO_PUBLIC_SCHOOLS where elementary__middle__or_high_school = 'ES'\
group by elementary__middle__or_high_school;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


number
462


Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [31]:
%sql select max(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


1
99


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [33]:
%sql select school_id,name_of_school, safety_score from chicago_public_schools\
where safety_score = (select max(safety_score) from chicago_public_schools);

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,safety_score
610038,Abraham Lincoln Elementary School,99
609799,Alexander Graham Bell Elementary School,99
610084,Annie Keller Elementary Gifted Magnet School,99
609820,Augustus H Burley Elementary School,99
610132,Edgar Allan Poe Elementary Classical School,99
609901,Edgebrook Elementary School,99
610073,Ellen Mitchell Elementary School,99
610066,James E McDade Elementary Classical School,99
609803,James G Blaine Elementary School,99
610033,LaSalle Elementary Language Academy,99


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [39]:
%sql select  school_id,name_of_school,average_student_attendance from CHICAGO_PUBLIC_SCHOOLS\
where average_student_attendance != 'None'\
order by average_student_attendance desc limit 10;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,average_student_attendance
609959,John Charles Haines Elementary School,98.40%
610217,James Ward Elementary School,97.80%
610132,Edgar Allan Poe Elementary Classical School,97.60%
609842,Rachel Carson Elementary School,97.60%
610329,Orozco Fine Arts & Sciences Elementary School,97.60%
610084,Annie Keller Elementary Gifted Magnet School,97.50%
610060,Andrew Jackson Elementary Language Academy,97.40%
610298,Lenart Elementary Regional Gifted Center,97.40%
610515,Disney II Magnet School,97.30%
610207,John H Vanderpoel Elementary Magnet School,97.20%


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [40]:
%sql select  school_id,name_of_school,average_student_attendance from CHICAGO_PUBLIC_SCHOOLS\
where average_student_attendance != 'None'\
order by average_student_attendance limit 5;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,average_student_attendance
609702,Richard T Crane Technical Preparatory High School,57.90%
609871,Barbara Vick Early Childhood & Family Center,60.90%
609736,Dyett High School,62.50%
609727,Wendell Phillips Academy High School,63.00%
610389,Orr Academy High School,66.30%


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [42]:
%sql select  school_id,name_of_school,replace(average_student_attendance,'%','') as att from CHICAGO_PUBLIC_SCHOOLS\
where average_student_attendance != 'None'\
order by average_student_attendance limit 5;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,att
609702,Richard T Crane Technical Preparatory High School,57.90
609871,Barbara Vick Early Childhood & Family Center,60.90
609736,Dyett High School,62.50
609727,Wendell Phillips Academy High School,63.00
610389,Orr Academy High School,66.30


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [44]:
%sql select  school_id,name_of_school,average_student_attendance from CHICAGO_PUBLIC_SCHOOLS\
where CAST(replace(average_student_attendance,'%','') as DOUBLE)< 70\
order by average_student_attendance limit 5;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,average_student_attendance
609702,Richard T Crane Technical Preparatory High School,57.90%
609871,Barbara Vick Early Childhood & Family Center,60.90%
609736,Dyett High School,62.50%
609727,Wendell Phillips Academy High School,63.00%
610389,Orr Academy High School,66.30%


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [50]:
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from CHICAGO_PUBLIC_SCHOOLS \
   group by Community_Area_Name;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


community_area_name,total_enrollment
ALBANY PARK,6864
ARCHER HEIGHTS,4823
ARMOUR SQUARE,1458
ASHBURN,6483
AUBURN GRESHAM,4175
AUSTIN,10933
AVALON PARK,1522
AVONDALE,3640
BELMONT CRAGIN,14386
BEVERLY,1636


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [53]:
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from CHICAGO_PUBLIC_SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


community_area_name,total_enrollment
OAKLAND,140
FULLER PARK,531
BURNSIDE,549
OHARE,786
LOOP,871


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [54]:
%sql select  school_id,name_of_school,safety_score from CHICAGO_PUBLIC_SCHOOLS\
order by safety_score limit 5;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,safety_score
609819,Edmond Burke Elementary School,1
610108,Luke O'Toole Elementary School,5
610202,George W Tilton Elementary School,6
609927,Foster Park Elementary School,11
609712,Emil G Hirsch Metropolitan High School,13


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score \
FROM schools \
ORDER BY safety_score \
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [26]:
%sql select * from CHICAGO_PUBLIC_SCHOOLS limit 2;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


school_id,name_of_school,elementary__middle__or_high_school,street_address,city,state,zip_code,phone_number,link,network_manager,collaborative_name,adequate_yearly_progress_made_,track_schedule,cps_performance_policy_status,cps_performance_policy_level,healthy_school_certified,safety_icon,safety_score,family_involvement_icon,family_involvement_score,environment_icon,environment_score,instruction_icon,instruction_score,leaders_icon,leaders_score,teachers_icon,teachers_score,parent_engagement_icon,parent_engagement_score,parent_environment_icon,parent_environment_score,average_student_attendance,rate_of_misconducts__per_100_students_,average_teacher_attendance,individualized_education_program_compliance_rate,pk_2_literacy__,pk_2_math__,gr3_5_grade_level_math__,gr3_5_grade_level_read__,gr3_5_keep_pace_read__,gr3_5_keep_pace_math__,gr6_8_grade_level_math__,gr6_8_grade_level_read__,gr6_8_keep_pace_math_,gr6_8_keep_pace_read__,gr_8_explore_math__,gr_8_explore_read__,isat_exceeding_math__,isat_exceeding_reading__,isat_value_add_math,isat_value_add_read,isat_value_add_color_math,isat_value_add_color_read,students_taking__algebra__,students_passing__algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),net_change_explore_and_plan,11th Grade Average ACT (2011),net_change_plan_and_act,college_eligibility__,graduation_rate__,college_enrollment_rate__,college_enrollment,general_services_route,freshman_on_track_rate__,x_coordinate,y_coordinate,latitude,longitude,community_area_number,community_area_name,ward,police_district,location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99,Very Strong,99,Strong,74,Strong,66,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54,Strong,66,Strong,74,Very Strong,84,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"


In [57]:
%sql select * from CHICAGO_SOCIOECONOMIC_DATA limit 2;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


index,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0


In [62]:
%sql select  C2.community_area_name,C2.hardship_index from CHICAGO_PUBLIC_SCHOOLS C1,\
CHICAGO_SOCIOECONOMIC_DATA C2\
where LCASE(C2.community_area_name) = LCASE(C1.community_area_name) and C1.college_enrollment = 4368;

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


community_area_name,hardship_index
North Center,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [64]:
%sql select  C2.community_area_name,C2.hardship_index from CHICAGO_PUBLIC_SCHOOLS C1,\
CHICAGO_SOCIOECONOMIC_DATA C2\
where LCASE(C2.community_area_name) = LCASE(C1.community_area_name)\
and C1.college_enrollment = (select max(college_enrollment) from CHICAGO_PUBLIC_SCHOOLS);

 * ibm_db_sa://DLV46630:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=ssl
Done.


community_area_name,hardship_index
North Center,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2021-07-09        | 2.4     | Malika     | Updated connection string  |
| 2021-05-19        | 2.3     | Lakshmi Holla |Updated question     |
| 2021-04-20        | 2.2     | Malika     | Added the libraries                |
| 2020-11-27        | 2.1     | Sannareddy Ramesh    | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
